In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load function into library

In [23]:
from cobos.tokens import ibm_token, group_instance, group_instance2
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(token=ibm_token, channel="ibm_quantum", instance=group_instance2, overwrite=True)

In [24]:
from qiskit_ibm_catalog import QiskitServerless, QiskitFunction

function_name = "Z2chain_postselection_calibration_and_run"
entry_point = "serverless_submit.py"
working_dir = "../../z2chain/serverless/"

overwrite = True

serverless = QiskitServerless(token=ibm_token)
functions = serverless.list()
found_funcs = [func for func in functions if func.title == function_name]
if found_funcs and not overwrite:
    qiskit_function = found_funcs[-1]
else:
    qiskit_function = QiskitFunction(title=function_name, entrypoint=entry_point, working_dir=working_dir, dependencies=["pymatching"])
    serverless.upload(qiskit_function)
    functions = serverless.list()
    qiskit_function = [func for func in functions if func.title == function_name][-1]

print(serverless.list())

[QiskitFunction(Z2chain postselection calibration and run), QiskitFunction(Z2chain_postselection_calibration_and_run)]


# Simulation settings

In [25]:
from utils.serverless import FunctionDB
from utils.hexec import ExecDB

debug = True

L_arr = [16]
J_arr = [1]
h_arr = [0.05]
lamb_arr = [0.7]
g_arr = [8]

# device = "ibm_kyiv"
# device = "ibm_brisbane"
device = "ibm_fez"
# device = "ibm_sherbrooke"
# device = "ibm_torino"
# device = "ibm_marrakesh"

x_basis = False

fractional_gates = False

particle_pair_left_positions = [L//2 - 1 for L in L_arr]
particle_pair_length = 1

final_time = 8
steps = 35
max_layers = 50

shots = 75000

odr = True
flip_threshold = 2

optimization_level = 3

sampler_options = {
    "default_shots": shots,
    "execution": {
        "init_qubits": True,
        "meas_type": "classified"
    },
    "dynamical_decoupling": {
        "enable": True,
        "sequence_type": "XY4"
    },
    "twirling": {
        "enable_gates": True,
        "enable_measure": True,
        "num_randomizations": shots//100,
        "shots_per_randomization": 100
    }
}

execution_database = "z2_chain_jobs_postselection_no_eplg.json"
function_database = "z2_chain_functions_postselection_no_eplg.json"
jobs_result_folder = execution_database.split(".")[0] + "/"
job_db = ExecDB(execution_database)
func_db = FunctionDB(function_database)

circ_folder_path = "Circuits/"

# Circuit execution

## Send jobs to hardware

In [26]:
from itertools import product

func_jobs = []

for i, (L, J, h, lamb, g) in enumerate(product(L_arr, J_arr, h_arr, lamb_arr, g_arr)):
    func_kwargs = {
        "chain_length": L,
        "J": J,
        "lamb": lamb,
        "particle_pair_left_position": particle_pair_left_positions[i],
        "particle_pair_length": particle_pair_length,
        "final_time": final_time,
        "steps": steps,
        "max_layers": max_layers,
        "backend": device,
        "optimization_level": optimization_level,
        "sampler_options": sampler_options,
        "g": g,
        "layout": None,
        "odr": odr,
        "x_basis": x_basis,
        "flip_threshold": flip_threshold,
        "debug": debug
    }
    this_job = qiskit_function.run(**func_kwargs)
    # func_db.add(func_kwargs, this_job.job_id)
    func_jobs.append(this_job)

In [27]:
serverless.jobs()

[<Job | deb26f31-f6fc-455a-a24b-bf648359a45c>,
 <Job | b637e243-f9d5-4777-aabc-5d01ce69d870>,
 <Job | b6bb0277-93f6-4cd1-97a0-90c0887e8918>,
 <Job | affb0668-881d-41c2-8733-594e84eba744>,
 <Job | d2fde3c3-1000-45eb-a10a-d0ab0255ae98>,
 <Job | af1ecb90-a29f-4bf3-8bea-477ffd86a992>,
 <Job | c5047e6d-fc44-49cc-86d3-22c6d1599cbe>,
 <Job | 8147e356-ed4b-460a-b0ff-b692bd006a8d>]

In [33]:
print(serverless.jobs()[0].status())
print(serverless.jobs()[0].logs())

ERROR
Job deb26f31-f6fc-455a-a24b-bf648359a45c failed due to an internal error.


## Load last functions jobs & update

In [ ]:
update_all = True

this_job.update_jobdb(job_db, serverless,)

for i, (L, J, h, lamb, g) in enumerate(product(L_arr, J_arr, h_arr, lamb_arr, g_arr)):
    func_kwargs = {
        "chain_length": L,
        "J": J,
        "lamb": lamb,
        "particle_pair_left_position": particle_pair_left_positions[i],
        "particle_pair_length": particle_pair_length,
        "final_time": final_time,
        "steps": steps,
        "max_layers": max_layers,
        "backend": device,
        "optimization_level": optimization_level,
        "sampler_options": sampler_options,
        "g": g,
        "layout": None,
        "odr": odr,
        "x_basis": x_basis,
        "flip_threshold": flip_threshold,
        "debug": debug
    }
    this_job = func_db.search_by_params(func_kwargs, serverless, limit=1)
    print(f"L = {L} / J = {J:.02f} / h = {lamb:.02f} / g = {g:.02f}")
    if len(this_job) > 0:
        print(f"\tStatus: {this_job.status()}")
        print(f"\tLogs:\n{this_job.logs()}")
    else:
        print("Job not found")